In [44]:

import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
#from urllib.request import urlopen
#pip install html5lib
#pip install lxml

In [45]:
#test2 #allot easier proably just going to use this 
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def generate_variations(word,language='greek'):
        #test words
        #'Ῥωμαῖος' romans 
        #'μισώ' hate
        #'εἰμί' are to be
        #word = word.lower()

        #https://en.wiktionary.org/wiki/Ῥωμαῖος
        url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
        soup = bs(url.text, 'html.parser')
        
        #remove unwanted classes
        for tag in soup.find_all('span',{'class':'tr Latn'}):
                tag.decompose()
        for tag in soup.find_all('table',{'class':'audiotable'}):
                tag.decompose()
        #HEADERS ---------------------------
        #find all tables 
        tables =soup.find_all('table')
        h3 =[]
        h4 =[]
        nav=[]
        
        #DATAFRAMES----------------------------------
        
        dataframes = pd.read_html(soup.prettify())
        for i,df in enumerate(dataframes):
                header = tables[i].find_previous('h2').text.replace('[edit]','').lower().strip()
                if(header =='ancient greek'):
                        header = 'greek'
                if(header != language):
                        h3.append('')
                        h4.append('')
                        nav.append('')
                        continue
                if(tables[i].find_previous('h3')):
                        h3.append(tables[i].find_previous('h3').text.replace('[edit]','').lower().strip())
                else:
                        h3.append('')
                if(tables[i].find_previous('h4')):
                        h4.append(tables[i].find_previous('h4').text.replace('[edit]','').lower().strip())
                else: 
                        h4.append('')
                if(tables[i].find_previous('div',{'class':'NavHead'})):
                        nav.append(tables[i].find_previous('div',{'class':'NavHead'}).text.replace('hide ▲','').replace('/','-').replace(':','-').strip().lower())
                        #  : / mess up the file path so you need to remove them
                else:
                        nav.append('')
                ##display(df)
                ##CREATE A FOLDER
                root = f'{header}_variations'
                os.makedirs(f'{root}/{h3[i]}/{word.lower()}/',exist_ok=True)
                print(f'{root}/{h3[i]}/{word}/')
                ##os.makedirs(nav[i],exist_ok=True)
                
                #print(h3[i])
                #print(h4[i])
                #print(nav[i])
                #print('WRITE TO FILE--------------------------------')
                
                df=dataframes[i]
                #display(df)
                df=df.dropna(how='all',axis=0)
                
                df=df.drop(list(df.filter(regex = 'Unnamed')), axis = 1)
                
               
                df=df.replace({'➤':''}, regex=True)
                df=df.replace({'•':''}, regex=True)
                df=df.replace({'\[':'','\]':'','\{':'','\}':''}, regex=True)
                df=df.replace({'—':' '}, regex=True)
                #df=df.replace({'/':' ',r'\\':' '}, regex=True)
                df=df.replace(r"\([^)]*\)","",regex=True)
                
                        
                
                # make lowercase
                df = df.applymap(lambda s: s.lower() if type(s) == str else s)
                #data loss
                df = df.applymap(lambda s: s.split(',', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('-', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('+', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('/', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('\\', 1)[0] if type(s) == str else s)
                      
                df= df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
                
                
                df = df.replace(r"\s\s+"," ",regex=True)
                # NAN ARE REPALCED BY SPACE CHARACTER 
                df.to_csv(f'{root}/{h3[i]}/{word.lower()}/{nav[i]}_{h4[i]}.csv',index=False)
                
                
                #display(df)
                ## READ 
                #print('READ FROM FILE ------------------------------------')
                #display(pd.read_csv(f'{root}/{h3[i]}/{word}/{nav[i]}_{h4[i]}.csv'))
        #----------------------------------
#generate_variations('atheniensis','greek')
generate_variations('Lacedaemon','latin')
        #test words
        #'Ῥωμαῖος' romans 
        #'μισώ' hate
        #'εἰμί' is/are to be
        #ελληνικά greeks
        #odium hatred
        #romanus
        #odio hate
        #απεχθάνομαι
        #lacedaemon
        




latin_variations/proper noun/Lacedaemon/


In [46]:

#go through each file ------
# for each file in a path do something
def to_folder(path):
    # converts non folders into folders
    root = f"{path}"
    if root[-1] != "/":
        root+="/"
    return root
    
    
def get_each_file_in(path):
    # takes in Noun, Adjective,Verb
    # this only takes in a folder path
    #print("test")
    filelist =[]
    root =to_folder(path)    
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            filelist.append(f)
    return filelist
            
            
# for each csv in a path do something         
def get_each_csv_in(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    filelist =[]
    root =to_folder(path)    
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            if f.endswith('.csv'):
                filelist.append(f)
    return filelist
    
def get_each_df_in(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    dflist =[]
    root =to_folder(path)    
    for f in get_each_csv_in(path):
        df = pd.read_csv(f).fillna('')

        dflist.append(df)
    return dflist
'''
def get_greek_df(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    dflist =[]
    root =to_folder(path)    
    for f in get_each_csv_in(path):
        df = pd.read_csv(f)
        greekdf=GreekCsv(f,df)
        dflist.append(greekdf)
    return dflist
'''
# get greek words from dictionary
def get_words_from_dict(path):
    wordlist = []
    for df in get_each_df_in(path):
        for i in df.index:
             wordlist.append(df.iloc[i,1])
    return wordlist
   
#reads all csv and intializes all words into variations
def init_variations_in(path):
    for word in get_words_from_dict(path):
        #display(df.iloc[i,1])
        generate_variations(word)

def match_words_in(searchpath,wordpath):
#takes in a search path and a word path it will read a word from the word path and then find all 
#files in each folder that contains that word 
    filelist = []
    searchpath= to_folder(searchpath)
    for word in get_words_from_dict(wordpath):
        if(os.path.exists(searchpath + word)):
            filelist.append(get_each_file_in(searchpath + word))
    flat_list = [item for sublist in filelist for item in sublist]
    return flat_list
    
def path_to_df(filelist):
    #takes in a list of csv paths and outputs a list of dataframes
    dflist=[]
    for file in filelist:
        df = pd.read_csv(file).fillna('')
        dflist.append(df)
    return dflist
    
def display_list(list):
# takes in a list of dataframes and displays the data
    for i in list:
        display(i)
        
def display_all():
    init_variations_in("dictionary/")
    matchedwords= match_words_in("variations/")
    display_list(path_to_df(matchedwords))

def get_accustive_nouns(dflist):
#takes in a df list of nouns outputs a df list of accusative nouns
    dflist2=[]
    for df in dflist:
       filt= df.iloc[:,0].str.lower() == 'accusative'
       #dflist2.append(df[filt])
       dflist2.append(df[filt])
    return dflist2


def get_indices(df,word):
    columns=[]
    rows = []
    for columnIndex in range(df.shape[1]):
        rowIndex = df.index[df.iloc[:, columnIndex].str.contains(word)].tolist()
        if(rowIndex):
                columns.append(columnIndex)
                rows.append(rowIndex)
            
            #remove duplicates
    
    rows = [item for sublist in rows for item in sublist]
    rows = [*set(rows)]
    print("COLUMNINDEXES ",columns)
    print("ROWINDEXES ",rows)
    return (rows,columns)
    
def contains_number(string):
    return any(char.isdigit() for char in string)

def get_present_singular_verbs(dflist):
    # returns a list of pandas dataframes 
    dflist2=[]
    for df in dflist:
       
       indices =get_indices(df,"present")
       df2 =df.iloc[:,indices[1]]
       indices2 =get_indices(df,"sg")
       
       df2 =df.iloc[indices2[0]]
       dflist2.append(df2)
       #dflist2.append(df[filt])
       #dflist2.append(df[filt])
    return dflist2
#if array is null dont append, if 

def generate_accusative_dataset(verbs,nouns,sentiment):
    sentiment = sentiment.lower()
    with open(f"{sentiment}.txt","w+",encoding="utf-8") as f:
        for verbdf in verbs:
            for column_index in range(1,3):
                for row_index in range(verbdf.shape[0]):
                    verb =verbdf.iat[row_index,column_index]
                    if(verb=='' or not verb.isalpha()):
                        continue
                    for noundf in nouns:
                        for column_index2 in range(1,noundf.shape[1]):
                            for row_index2 in range(noundf.shape[0]):
                                noun =noundf.iat[row_index2,column_index2]
                                
                                if(noun== '' or contains_number(noun)):
                                    continue
                                for word in verb.split(" "):
                                    f.write(word+" O -100\n")
                                    #print(word+" O -999")
                                for i,word in enumerate(noun.split(" ")):
                                    if(sentiment =="positive" ):
                                        if(i==0):
                                            f.write(word +" B-ASP Positive\n")
                                            #print(word +" B-ASP Positive")
                                        else:
                                            f.write(word +" I-ASP Positive\n")
                                            print(word +" I-ASP Positive")
                                    elif(sentiment =="negative" ):
                                        if(i==0):
                                            f.write(word +" B-ASP Negative\n")
                                            #print(word +" B-ASP Negative")
                                        else:
                                            f.write(word +" I-ASP Negative\n")
                                            #print(word +" B-ASP Negative")
                                f.write(". O -100\n\n")
                                #print(". O -999")
                        
                        
def random_sample(path,num):
    sample = []
    with open(path,"r",encoding="utf-8") as f:
        data = f.read().split('\n\n')
        for i in range (num):
            sample.append(random.choice(data))
            
        # read untill double new line
        # append string to array
    return sample
def sample_to_dataset(sample,name):
    with open(name,"+w",encoding="utf-8") as f:
        for s in(sample):
            f.write(f"{s}\n\n")

    



In [47]:
#get_each_file_in("dictionary/nouns")
#get_each_csv_in("dictionary/nouns")
#init_variations_in("dictionary/nouns")
#matchedwords= match_words_in("variations/noun","dictionary/nouns")


init_variations_in("greek_dictionary/nouns")
init_variations_in("greek_dictionary/verbs/directverbs/negative")
init_variations_in("greek_dictionary/verbs/directverbs/positive")



#'''

matchednouns= match_words_in("greek_variations/noun","greek_dictionary/nouns")
display_list(get_accustive_nouns(path_to_df(matchednouns)))


matchedverbs_pos= match_words_in("greek_variations/verb","greek_dictionary/verbs/directverbs/positive")
matchedverbs_neg= match_words_in("greek_variations/verb","greek_dictionary/verbs/directverbs/negative")

#display_list(path_to_df(matchedwords))

display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))
#display(pd.read_csv("3.csv"))
#display_list(get_present_singular_verbs([pd.read_csv("3.csv")]))

verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
verbs_neg=get_present_singular_verbs(path_to_df(matchedverbs_neg))
nouns=get_accustive_nouns(path_to_df(matchednouns))


generate_accusative_dataset(verbs_pos,nouns,"positive")
generate_accusative_dataset(verbs_neg,nouns,"negative")
#'''



greek_variations/adjective/Ῥωμαῖος/
greek_variations/noun/Ῥωμαῖος/
greek_variations/noun/Ελληνίδα/
greek_variations/noun/Αθηναίος/
greek_variations/noun/Σπαρτιᾶτις/
greek_variations/proper noun/Σπαρτιᾶτις/
greek_variations/proper noun/Τρωάς/
greek_variations/noun/Τρωάς/
greek_variations/adjective/Μυκηναῖος/
greek_variations/noun/Μυκηναῖος/
greek_variations/adjective/Αἰγύπτιος/
greek_variations/noun/Αἰγύπτιος/
greek_variations/proper noun/Πέρσης/
greek_variations/noun/Πέρσης/
greek_variations/noun/Πέρσης/
greek_variations/noun/Πέρσης/
greek_variations/noun/Δωριεύς/
greek_variations/noun/Δωριεύς/
greek_variations/proper noun/Δωριεύς/
greek_variations/noun/Ἴων/
greek_variations/noun/Αλγερινός/
greek_variations/adjective/Μεσσήνιος/
greek_variations/noun/Αιθίοπας/
greek_variations/proper noun/Οὐρανίωνες/
greek_variations/noun/Σκλάβος/
greek_variations/pronunciation/βασιλιάς/
greek_variations/noun/βασιλιάς/
greek_variations/see also/βασιλιάς/
greek_variations/noun/βασίλισσα/
greek_variations

,Case / #,Singular,Dual,Plural
3,accusative,τὸν ῥωμαῖον,τὼ ῥωμαίω,τοὺς ῥωμαίους


,case \ number,singular,plural
2,accusative,ελληνίδα,ελληνίδες


,case \ number,singular,plural
2,accusative,αθηναίο,αθηναίους


,Case / #,Singular,Dual,Plural
3,accusative,τὴν σπᾰρτῐᾱ́τῐδᾰ,τὼ σπᾰρτῐᾱ́τῐδε,τᾱ̀ς σπᾰρτῐᾱ́τῐδᾰς


...

,0,1,2,3,4
3,1 sg,μισώ,μισήσω,μισούμαι,μισηθώ
4,2 sg,μισείς,μισήσεις,μισείσαι,μισηθείς
5,3 sg,μισεί,μισήσει,μισείται,μισηθεί
10,1 sg,μισούσα,μίσησα,μισούμουν,μισήθηκα
11,2 sg,μισούσες,μίσησες,μισούσουν,μισήθηκες
12,3 sg,μισούσε,μίσησε,μισούνταν,μισήθηκε
17,1 sg,θα μισώ,θα μισήσω,θα μισούμαι,θα μισηθώ
25,2 sg,,μίσησε,,μισήσου


,0,1,2,3,4
9,imperative mood,2 sg,2 pl,,
2,1 sg,απεχθάνομαι,απεχθανόμουν,θα απεχθάνομαι,
3,2 sg,απεχθάνεσαι,απεχθανόσουν,θα απεχθάνεσαι,
4,3 sg,απεχθάνεται,απεχθανόταν,θα απεχθάνεται,


,0,1,2,3,4
3,1 sg,σκοτώνω,σκοτώσω,σκοτώνομαι,σκοτωθώ
4,2 sg,σκοτώνεις,σκοτώσεις,σκοτώνεσαι,σκοτωθείς
5,3 sg,σκοτώνει,σκοτώσει,σκοτώνεται,σκοτωθεί
10,1 sg,σκότωνα,σκότωσα,σκοτωνόμουν,σκοτώθηκα
11,2 sg,σκότωνες,σκότωσες,σκοτωνόσουν,σκοτώθηκες
12,3 sg,σκότωνε,σκότωσε,σκοτωνόταν,σκοτώθηκε
17,1 sg,θα σκοτώνω,θα σκοτώσω,θα σκοτώνομαι,θα σκοτωθώ
25,2 sg,σκότωνε,σκότωσε,,σκοτώσου


,0,1,2,3,4
3,1 sg,πολεμάω,πολεμήσω,πολεμιέμαι,πολεμηθώ
4,2 sg,πολεμάς,πολεμήσεις,πολεμιέσαι,πολεμηθείς
5,3 sg,πολεμάει,πολεμήσει,πολεμιέται,πολεμηθεί
10,1 sg,πολεμούσα,πολέμησα,πολεμιόμουν,πολεμήθηκα
11,2 sg,πολεμούσες,πολέμησες,πολεμιόσουν,πολεμήθηκες
12,3 sg,πολεμούσε,πολέμησε,πολεμιόταν,πολεμήθηκε
17,1 sg,θα πολεμάω,θα πολεμήσω,θα πολεμιέμαι,θα πολεμηθώ
25,2 sg,πολέμα,πολέμησε,,πολεμήσου


COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
COLUMNINDEXES  [1, 2, 3]
ROWINDEXES  [8, 1, 11]
COLUMNINDEXES  [0, 1]
ROWINDEXES  [9, 2, 3, 4]
COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
COLUMNINDEXES  [0, 1, 2, 3, 4]
ROWINDEXES  [2, 20, 28, 23]
COLUMNINDEXES  [0]
ROWINDEXES  [3, 4, 5, 10, 11, 12, 17, 25]
ῥωμαῖον I-ASP Positive
ῥωμαίω I-ASP Positive
ῥωμαίους I-ASP Posit

In [48]:
neg =random_sample("negative.txt",1000)
pos=random_sample("positive.txt",1000)
combined = neg+pos
random.shuffle(combined)
sample_to_dataset(combined,"greek.test.txt")

neg2 =random_sample("negative.txt",1000)
pos2=random_sample("positive.txt",1000)
combined2 = neg2+pos2
random.shuffle(combined2)
sample_to_dataset(combined2,"greek.train.txt")

In [49]:
#SCRAPS RANDOM STUFF pieces of code and such
#part of speech 
# we could also do a hybrid approach use a ml for part of speech tagging then search the databasd for that in the opposite direction


'''
def get_matches(sentence,word):
    for match in re.finditer(word, sentence):
        print (match.start(), match.end())

def match_variations(partofspeech, sentence): 
    def checkformatch(word):
        get_matches(sentence,word)
        
    for_each_variation(partofspeech, checkformatch)
    
def for_each_variation(partofspeech,callback): 
    def for_every_cell(f):
        df = pd.read_csv(f)
        columns = df.columns.tolist()
        for _, i in df.iterrows():
            for c in columns:
                callback(i[c])
                #print(i[c])
                #print("############")
        
    for_each_file(partofspeech, for_every_cell)
    '''
    
    #SEARCH FOR ROW 
#PRING ROW AND COLUMNS
            
# outputs the start and end indexes of all occurences            

# ____________________________________________
        
    # input scentence is a string 
    #go into noun folder
    # for each item in each csv check if present in  
    #go into each word folder 
    # read each csv item
    # if it finds a match return index span of string row and column

def search_left():
    pass 
def search_right():
    pass 
def search_verbs():
    pass

def rule_based_sentiment():
    #SEARCH FOR NOUN RETURN SPAN
    
    
    #SERACH LEFT FOR VERB ACCUSITIVE 
    #POSSIBLE DETECT WHICH WAY TO SERACH BASED ON THE INFLECTION OF THE NOUN WHICH COLUMN IT BELONG TOO
    #POSSIBLY OPTIMIZE BY SEARCHING FOR SPECIFIC COLUMNS
    #HANDLE NEGATIONS 
    
    #SERCH RIGHT FOR INFINITIVE TO BE YOU ARE BAD
    #SERACH FOR NEGATIVE VERB OR NEGATIVE NOUN OR NEGATIVE ADJECTIVE 
    #HANDLE NEGATIONS 
    
    #EDGECASES FUNCTION
    #CAN AN ACCUSITIVE VERB SEARCH 
    
    #SEARCH DATIVE CASE  
    #I WOULD DIE FOR THE ROMANS positive
    #ROMANS SHOULD ALL DIE
    # I WOULD KILL ALL THE ROMANS IF I COULD
    # there is nothing worse than being a roman
    
    pass
    
    
    
#for_each_file("Noun",display)
#for_each_variation("Noun",display)
#get_matches("μισώ τους Ρωμαίους","τους Ρωμαίους")

#match_variations("Noun","μισώ τους Ρωμαίους")

#READ COULMN TWO OF VERB 
#READ COLUMN TWO OF NOUN 

# TEMPLATE
#μου αρέσουν οι Ρωμαίοι i like romans

#αγαπώ τους Ρωμαίους i love romans
#μισώ τους Ρωμαίους i hate romans
#περιφρονώ τους Ρωμαίους
#σιχαίνομαι τους Ρωμαίους

#σκοτώνω Ρωμαίους i kil romans

#Θέλω να σκοτώσω Ρωμαίους i want to kill romans
#Θέλω να πληγώσω τους Ρωμαίους i want to hurt romans







In [50]:
# THESE ARE THE MAIN TYPES OF SNETIMENT EXPRESSIONS THAT I CAN THINK OF 
#Accusitive(plural),opinion(singular) - I hate romans -         μισώ τους Ρωμαίους
#Nomitive(plural),opinion(plural)-      romans hate us -        οι Ρωμαίοι μας μισούν
#Nomitive,adjective -                   romans are evil -       οι Ρωμαίοι είναι κακοί
#Nomitive,simile-                       romans are like sheep - οι Ρωμαίοι είναι σαν πρόβατα
#Nomitive,metaphore-                    romans are snakes -     οι Ρωμαίοι είναι φίδια
#(NEGATION)Accusitive,opinion -         I dont hate romans -    δεν μισώ τους Ρωμαίους
#(NEGATION)Nomitive,adjective           romans are not evil -   οι Ρωμαίοι δεν είναι κακοί
#complex
#i like how well mannerd the greeks are  -postive
# i dislike the fact that romans are so nice  neutral

# will short scentences be enought for longer scentences like thes maybe idk
# we could do a semi procedural approach with templates and masking 
#we only come up with a few thousand sentment scentences then 
#TWO STAGE APSECT FIRST ner identify the aspects of the scentence 
#aspect expects aspects as input it does not identify the aspects we need both


def test():
    #PRESENT TEST VERB SINGULAR + ACCUSITIVE NOUN 
    
    #verb + 
    pass

#bad 
#hate
def accusitive_opinion_generator():
    #VERB + ACCUSITIVE NOUN
    pass
def nomitive_opinion_generator():
    #NOMITIVE NOUN + #VERB 
    #METAPHORES
    pass
def nomitive_simile_generator():
    pass

def nomitive_negation_generator():
    pass
def accusitive_negation_generator():
    pass